# 2_data_preparation/02_create_four_year_dataset.ipynb
"""
# Create Four-Year Observation Dataset (2021-2025)

## Goal
Load a representative sample of observations from the last 4 years (2021-2025) 
and prepare it for ML modeling with proper station data joins.

## Fix Applied:
1. **Random Sampling**: Use `ORDER BY RAND()` instead of `ORDER BY start DESC` 
   to get a representative sample from the entire 4-year period
2. **Station Join Debugging**: Added diagnostics to understand join issues
3. **Data Validation**: Enhanced checks for temporal coverage

## Steps in This Notebook:
1. **Database Connection**: Connect to MariaDB
2. **Data Extraction**: Query RANDOM sample from last 4 years
3. **Join Operations**: Merge with station data + diagnostics
4. **Target Definition**: Create success/failure labels
5. **Feature Engineering**: Create temporal, geometric features
6. **Data Cleaning**: Handle missing values
7. **Dataset Export**: Save processed dataset

## Expected Output:
- A cleaned dataset of ~1M observations spanning 2021-2025
- Proper station data joins
- Seasonal representation (all 4 seasons)
"""

In [17]:
# ============================================================================
# IMPORT LIBRARIES
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')
import os
import time

print("="*80)
print("📡 CREATING FOUR-YEAR OBSERVATION DATASET (2021-2025)")
print("="*80)
print(f"Execution started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Working directory: {os.getcwd()}")

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

📡 CREATING FOUR-YEAR OBSERVATION DATASET (2021-2025)
Execution started: 2025-12-11 16:44:05
Working directory: d:\ELO 2\satnogs_project\satellite-pass-prediction\2_data_preparation


In [18]:
# ============================================================================
# STEP 1: DATABASE CONNECTION
# ============================================================================
print("\n" + "="*80)
print("STEP 1: DATABASE CONNECTION")
print("="*80)

# Database credentials
DB_USER = "root"
DB_PASSWORD = "123456789"
DB_HOST = "127.0.0.1"
DB_PORT = "3306"
DB_NAME = "satnogs"

# Create connection string
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

print(f"Connecting to database: {DB_NAME} on {DB_HOST}:{DB_PORT}")
print(f"User: {DB_USER}")

try:
    engine = create_engine(connection_string)
    
    # Test connection with a simple query
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1 as test"))
        test_value = result.fetchone()[0]
    
    if test_value == 1:
        print("✅ Database connection successful!")
    else:
        print("❌ Database connection test failed")
        raise ConnectionError("Could not connect to database")
        
except Exception as e:
    print(f"❌ Error connecting to database: {e}")
    print("Please check:")
    print("  1. Database is running")
    print("  2. Credentials are correct")
    print("  3. Port 3306 is open")
    raise SystemExit



STEP 1: DATABASE CONNECTION
Connecting to database: satnogs on 127.0.0.1:3306
User: root
✅ Database connection successful!


In [19]:
# ============================================================================
# STEP 2: DETERMINE DATE RANGE FOR LAST 4 YEARS
# ============================================================================
print("\n" + "="*80)
print("STEP 2: DETERMINING DATE RANGE")
print("="*80)

# Get the most recent observation date
print("Finding most recent observation date...")
start_time = time.time()

max_date_query = """
SELECT MAX(start) as latest_date FROM base_observation WHERE start IS NOT NULL
"""

try:
    max_date_df = pd.read_sql(max_date_query, engine)
    latest_date = pd.to_datetime(max_date_df['latest_date'].iloc[0])
    cutoff_date = latest_date - pd.DateOffset(years=4)
    
    print(f"✅ Latest observation date: {latest_date.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📅 4-year cutoff date: {cutoff_date.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️  Time period: {cutoff_date.strftime('%Y-%m-%d')} to {latest_date.strftime('%Y-%m-%d')}")
    
    # Get total count for the 4-year period
    count_query = f"""
    SELECT COUNT(*) as total_count 
    FROM base_observation 
    WHERE start >= '{cutoff_date.strftime('%Y-%m-%d')}'
      AND max_altitude IS NOT NULL
      AND status IS NOT NULL
      AND ground_station_id IS NOT NULL
    """
    
    count_df = pd.read_sql(count_query, engine)
    total_count = count_df['total_count'].iloc[0]
    
    print(f"📊 Total observations in last 4 years: {total_count:,}")
    
    # Calculate sampling rate needed for 1M rows
    sample_limit = 1000000
    if total_count > sample_limit:
        sampling_rate = sample_limit / total_count * 100
        print(f"📊 Sampling {sample_limit:,} rows ({sampling_rate:.2f}% of total)")
    else:
        sample_limit = total_count
        print(f"📊 Will load all {total_count:,} observations")
    
except Exception as e:
    print(f"❌ Error determining date range: {e}")
    raise SystemExit

print(f"⏱️  Date determination completed in {time.time() - start_time:.2f}s")


STEP 2: DETERMINING DATE RANGE
Finding most recent observation date...
✅ Latest observation date: 2025-11-12 09:59:41
📅 4-year cutoff date: 2021-11-12 09:59:41
⏱️  Time period: 2021-11-12 to 2025-11-12
📊 Total observations in last 4 years: 7,560,410
📊 Sampling 1,000,000 rows (13.23% of total)
⏱️  Date determination completed in 97.79s


In [20]:
# ============================================================================
# STEP 3: EXTRACT RANDOM SAMPLE FROM 4-YEAR PERIOD
# ============================================================================
print("\n" + "="*80)
print("STEP 3: EXTRACTING RANDOM SAMPLE FROM 4-YEAR PERIOD")
print("="*80)

print(f"Querying RANDOM sample of {sample_limit:,} observations from 2021-2025...")
print("Using ORDER BY RAND() for representative sampling across the entire period.")

start_time = time.time()

# Query for RANDOM sample from 4-year period
observation_query = f"""
SELECT 
    -- Core observation data
    id,
    start,
    end,
    status,
    waterfall_status,
    vetted_status,
    max_altitude,
    rise_azimuth,
    set_azimuth,
    ground_station_id,
    sat_id,
    
    -- Additional potentially useful fields
    archive_url,
    archived,
    experimental,
    client_version
    
FROM base_observation 
WHERE start >= '{cutoff_date.strftime('%Y-%m-%d')}'
  AND max_altitude IS NOT NULL  -- Critical feature
  AND status IS NOT NULL        -- Need status for target variable
  AND ground_station_id IS NOT NULL  -- Need station ID for joining
ORDER BY RAND()  -- KEY FIX: Random sampling for temporal distribution
LIMIT {sample_limit}
"""

print(f"Executing query: Random sample of {sample_limit:,} rows from 4-year period...")
try:
    obs_df = pd.read_sql(observation_query, engine)
    print(f"✅ Observations loaded: {len(obs_df):,} rows")
    print(f"⏱️  Load time: {time.time() - start_time:.2f}s")
    
    # Check date range of loaded data
    if not obs_df.empty:
        obs_df['start'] = pd.to_datetime(obs_df['start'])
        min_date = obs_df['start'].min()
        max_date = obs_df['start'].max()
        date_range_days = (max_date - min_date).days
        
        print(f"📅 Loaded data range: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")
        print(f"📅 Duration: {date_range_days} days ({date_range_days/365:.1f} years)")
        
        # Check year distribution
        obs_df['year'] = obs_df['start'].dt.year
        year_dist = obs_df['year'].value_counts().sort_index()
        print("\n📊 Year distribution in sample:")
        for year, count in year_dist.items():
            percentage = count / len(obs_df) * 100
            print(f"  {year}: {count:8,} rows ({percentage:6.2f}%)")
        
except Exception as e:
    print(f"❌ Error loading observations: {e}")
    print("Trying alternative query...")
    
    # Alternative: Use stratified sampling by year
    year_sampling_query = f"""
    WITH yearly_counts AS (
        SELECT YEAR(start) as obs_year, COUNT(*) as year_count
        FROM base_observation 
        WHERE start >= '{cutoff_date.strftime('%Y-%m-%d')}'
          AND max_altitude IS NOT NULL
          AND status IS NOT NULL
          AND ground_station_id IS NOT NULL
        GROUP BY YEAR(start)
    )
    SELECT o.*
    FROM base_observation o
    JOIN yearly_counts yc ON YEAR(o.start) = yc.obs_year
    WHERE o.start >= '{cutoff_date.strftime('%Y-%m-%d')}'
      AND o.max_altitude IS NOT NULL
      AND o.status IS NOT NULL
      AND o.ground_station_id IS NOT NULL
      AND RAND() < {sample_limit} / (SELECT SUM(year_count) FROM yearly_counts)
    LIMIT {sample_limit}
    """
    
    try:
        obs_df = pd.read_sql(year_sampling_query, engine)
        print(f"✅ Loaded stratified sample: {len(obs_df):,} rows")
    except Exception as e2:
        print(f"❌ Failed to load data: {e2}")
        raise SystemExit

print("\nObservation data preview:")
print(f"Shape: {obs_df.shape}")
print(f"Columns: {list(obs_df.columns)}")


STEP 3: EXTRACTING RANDOM SAMPLE FROM 4-YEAR PERIOD
Querying RANDOM sample of 1,000,000 observations from 2021-2025...
Using ORDER BY RAND() for representative sampling across the entire period.
Executing query: Random sample of 1,000,000 rows from 4-year period...
✅ Observations loaded: 1,000,000 rows
⏱️  Load time: 2037.08s
📅 Loaded data range: 2021-11-12 to 2025-11-12
📅 Duration: 1461 days (4.0 years)

📊 Year distribution in sample:
  2021:   31,157 rows (  3.12%)
  2022:  225,327 rows ( 22.53%)
  2023:  238,351 rows ( 23.84%)
  2024:  262,490 rows ( 26.25%)
  2025:  242,675 rows ( 24.27%)

Observation data preview:
Shape: (1000000, 16)
Columns: ['id', 'start', 'end', 'status', 'waterfall_status', 'vetted_status', 'max_altitude', 'rise_azimuth', 'set_azimuth', 'ground_station_id', 'sat_id', 'archive_url', 'archived', 'experimental', 'client_version', 'year']


In [21]:
# ============================================================================
# STEP 4: LOAD STATION DATA WITH BETTER FILTERING
# ============================================================================
print("\n" + "="*80)
print("STEP 4: LOADING STATION DATA")
print("="*80)

print("Loading ground station metadata...")
start_time = time.time()

# First, let's check what ground_station_id values we have in our observation sample
unique_ground_station_ids = obs_df['ground_station_id'].unique()
print(f"Unique ground_station_id values in our sample: {len(unique_ground_station_ids)}")

# Load ALL stations first, not just active ones
station_query = f"""
SELECT 
    id as station_id,
    name as station_name,
    lat as station_lat,
    lng as station_lng,
    alt as station_alt,
    horizon,
    qthlocator as grid_square,
    description as station_description,
    status as station_status,
    last_seen
FROM base_station
WHERE id IN ({','.join([str(int(x)) for x in unique_ground_station_ids if not pd.isna(x)])})
"""

try:
    station_df = pd.read_sql(station_query, engine)
    print(f"✅ Stations loaded: {len(station_df):,} stations matching our observations")
    print(f"⏱️  Load time: {time.time() - start_time:.2f}s")
    
    # Check if we got all stations
    matched_ids = set(station_df['station_id'].astype(int).tolist())
    requested_ids = set([int(x) for x in unique_ground_station_ids if not pd.isna(x)])
    
    print(f"\n📊 Station matching statistics:")
    print(f"  Observation station IDs requested: {len(requested_ids)}")
    print(f"  Station IDs found in database: {len(matched_ids)}")
    print(f"  Match rate: {len(matched_ids)/len(requested_ids)*100:.1f}%")
    
    if len(requested_ids) - len(matched_ids) > 0:
        missing_ids = list(requested_ids - matched_ids)[:10]
        print(f"  Sample missing station IDs: {missing_ids}")
    
except Exception as e:
    print(f"⚠️  Error loading specific stations: {e}")
    print("Loading all stations as fallback...")
    
    # Fallback: Load all stations
    station_query_all = """
    SELECT 
        id as station_id,
        name as station_name,
        lat as station_lat,
        lng as station_lng,
        alt as station_alt,
        horizon,
        qthlocator as grid_square,
        description as station_description,
        status as station_status,
        last_seen
    FROM base_station
    WHERE lat IS NOT NULL AND lng IS NOT NULL
    """
    
    station_df = pd.read_sql(station_query_all, engine)
    print(f"✅ Loaded all stations: {len(station_df):,} stations")

print("\nStation data preview:")
print(f"Shape: {station_df.shape}")
print(f"Columns: {list(station_df.columns)}")


STEP 4: LOADING STATION DATA
Loading ground station metadata...
Unique ground_station_id values in our sample: 1556
✅ Stations loaded: 1,556 stations matching our observations
⏱️  Load time: 0.12s

📊 Station matching statistics:
  Observation station IDs requested: 1556
  Station IDs found in database: 1556
  Match rate: 100.0%

Station data preview:
Shape: (1556, 10)
Columns: ['station_id', 'station_name', 'station_lat', 'station_lng', 'station_alt', 'horizon', 'grid_square', 'station_description', 'station_status', 'last_seen']


In [22]:
# ============================================================================
# STEP 5: JOIN OBSERVATIONS WITH STATION DATA
# ============================================================================
print("\n" + "="*80)
print("STEP 5: JOINING OBSERVATIONS WITH STATIONS")
print("="*80)

print(f"Before join: {len(obs_df):,} observations")
print(f"Available stations: {len(station_df):,}")

# Ensure data types match for joining
obs_df['ground_station_id'] = obs_df['ground_station_id'].astype(int)
station_df['station_id'] = station_df['station_id'].astype(int)

# Perform the join
start_time = time.time()

df_merged = obs_df.merge(
    station_df,
    left_on='ground_station_id',
    right_on='station_id',
    how='left',  # Keep all observations even if station not found
    indicator=True  # Track join status
)

print(f"After join: {len(df_merged):,} observations")
print(f"⏱️  Join completed in {time.time() - start_time:.2f}s")

# Analyze join quality
join_stats = df_merged['_merge'].value_counts()
print("\n📊 Join statistics:")
for merge_type, count in join_stats.items():
    percentage = count / len(df_merged) * 100
    print(f"  {merge_type:15}: {count:8,} rows ({percentage:6.2f}%)")

# Calculate expected vs actual join rate
expected_join_rate = len(station_df) / len(unique_ground_station_ids) * 100
print(f"\nExpected join rate (based on station lookup): {expected_join_rate:.1f}%")
print(f"Actual join rate: {join_stats.get('both', 0)/len(df_merged)*100:.1f}%")

# Drop the merge indicator column
df_merged = df_merged.drop(columns=['_merge'])

print(f"\nMerged dataset shape: {df_merged.shape}")
print(f"Columns in merged dataset: {len(df_merged.columns)}")


STEP 5: JOINING OBSERVATIONS WITH STATIONS
Before join: 1,000,000 observations
Available stations: 1,556
After join: 1,000,000 observations
⏱️  Join completed in 1.29s

📊 Join statistics:
  both           : 1,000,000 rows (100.00%)
  left_only      :        0 rows (  0.00%)
  right_only     :        0 rows (  0.00%)

Expected join rate (based on station lookup): 100.0%
Actual join rate: 100.0%

Merged dataset shape: (1000000, 26)
Columns in merged dataset: 26


In [23]:
# ============================================================================
# STEP 6: DEFINE TARGET VARIABLE
# ============================================================================
print("\n" + "="*80)
print("STEP 6: DEFINING TARGET VARIABLE")
print("="*80)

print("Analyzing status codes in 4-year data...")
status_dist = df_merged['status'].value_counts().head(10)
print("\nTop 10 status codes in 4-year data:")
for status_code, count in status_dist.items():
    percentage = count / len(df_merged) * 100
    print(f"  Status {status_code:6}: {count:8,} rows ({percentage:6.2f}%)")

# Create target mapping
def map_status_to_target(status):
    """Map status code to target variable."""
    if pd.isna(status):
        return np.nan
    
    status = int(status) if not pd.isna(status) else status
    
    # Success cases
    if status == 0:
        return 1  # Success
    
    # Failure cases
    elif status in [-100, -1000]:
        return 0  # Failure
    
    # Ambiguous/unknown cases
    else:
        return np.nan  # Will filter these out

# Create binary target
df_merged['target_success'] = df_merged['status'].apply(map_status_to_target)

# Also create detailed status category for analysis
def map_status_to_category(status):
    """Map status code to descriptive category."""
    if pd.isna(status):
        return 'unknown'
    
    status = int(status) if not pd.isna(status) else status
    
    if status == 0:
        return 'success'
    elif status == -100:
        return 'failure'
    elif status == -1000:
        return 'severe_failure'
    elif status == 100:
        return 'in_progress'
    elif status > 0:
        return f'positive_{status}'
    else:
        return f'negative_{abs(status)}'

df_merged['status_category'] = df_merged['status'].apply(map_status_to_category)

# Analyze target distribution
print("\n🎯 Target variable distribution:")
target_dist = df_merged['target_success'].value_counts(dropna=False)

total_obs = len(df_merged)
for target_val, count in target_dist.items():
    if pd.isna(target_val):
        label = 'Ambiguous (will be filtered out)'
    else:
        label = 'Success' if target_val == 1 else 'Failure'
    
    percentage = count / total_obs * 100
    print(f"  {label:30}: {count:8,} rows ({percentage:6.2f}%)")

# Calculate overall success rate
success_count = target_dist.get(1, 0)
failure_count = target_dist.get(0, 0)
valid_obs = success_count + failure_count

if valid_obs > 0:
    success_rate = success_count / valid_obs * 100
    print(f"\n📈 Success rate (excluding ambiguous): {success_rate:.2f}%")
    print(f"   ({success_count:,} successes / {valid_obs:,} valid observations)")


STEP 6: DEFINING TARGET VARIABLE
Analyzing status codes in 4-year data...

Top 10 status codes in 4-year data:
  Status    100:  421,988 rows ( 42.20%)
  Status      0:  285,876 rows ( 28.59%)
  Status   -100:  202,384 rows ( 20.24%)
  Status  -1000:   89,752 rows (  8.98%)

🎯 Target variable distribution:
  Ambiguous (will be filtered out):  421,988 rows ( 42.20%)
  Failure                       :  292,136 rows ( 29.21%)
  Success                       :  285,876 rows ( 28.59%)

📈 Success rate (excluding ambiguous): 49.46%
   (285,876 successes / 578,012 valid observations)


In [24]:
# ============================================================================
# STEP 7: CREATE TEMPORAL FEATURES
# ============================================================================
print("\n" + "="*80)
print("STEP 7: CREATING TEMPORAL FEATURES")
print("="*80)

print("Converting timestamps and creating temporal features...")
start_time = time.time()

# Convert timestamps
df_merged['start'] = pd.to_datetime(df_merged['start'], errors='coerce')
df_merged['end'] = pd.to_datetime(df_merged['end'], errors='coerce')

# Calculate duration in seconds
df_merged['duration_seconds'] = (df_merged['end'] - df_merged['start']).dt.total_seconds()

# Extract time components
df_merged['hour_of_day'] = df_merged['start'].dt.hour
df_merged['day_of_week'] = df_merged['start'].dt.dayofweek  # Monday=0, Sunday=6
df_merged['month'] = df_merged['start'].dt.month
df_merged['year'] = df_merged['start'].dt.year
df_merged['day_of_year'] = df_merged['start'].dt.dayofyear
df_merged['week_of_year'] = df_merged['start'].dt.isocalendar().week

# Create time of day categories
def categorize_time_of_day(hour):
    """Categorize hour into time of day periods."""
    if pd.isna(hour):
        return 'unknown'
    elif 0 <= hour < 6:
        return 'night'
    elif 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    else:
        return 'evening'

df_merged['time_of_day'] = df_merged['hour_of_day'].apply(categorize_time_of_day)

# Create season based on month
def get_season(month):
    """Convert month to season (northern hemisphere)."""
    if pd.isna(month):
        return 'unknown'
    elif month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

df_merged['season'] = df_merged['month'].apply(get_season)

print(f"✅ Temporal features created in {time.time() - start_time:.2f}s")

# Display temporal distribution
print(f"\n📅 Temporal distribution:")
print(f"   - Date range: {df_merged['start'].min().strftime('%Y-%m-%d')} to {df_merged['start'].max().strftime('%Y-%m-%d')}")
print(f"   - Years: {sorted(df_merged['year'].unique())}")
print(f"   - Months: {sorted(df_merged['month'].unique())}")
print(f"   - Seasons: {sorted(df_merged['season'].unique())}")
print(f"   - Time of day: {sorted(df_merged['time_of_day'].unique())}")

# Check if we have all seasons
expected_seasons = ['winter', 'spring', 'summer', 'fall']
actual_seasons = df_merged['season'].unique()
missing_seasons = set(expected_seasons) - set(actual_seasons)

if missing_seasons:
    print(f"⚠️  Missing seasons: {missing_seasons}")
else:
    print("✅ All seasons represented in data")


STEP 7: CREATING TEMPORAL FEATURES
Converting timestamps and creating temporal features...
✅ Temporal features created in 3.24s

📅 Temporal distribution:
   - Date range: 2021-11-12 to 2025-11-12
   - Years: [np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025)]
   - Months: [np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12)]
   - Seasons: ['fall', 'spring', 'summer', 'winter']
   - Time of day: ['afternoon', 'evening', 'morning', 'night']
✅ All seasons represented in data


In [25]:
# ============================================================================
# STEP 8: CREATE GEOMETRIC FEATURES
# ============================================================================
print("\n" + "="*80)
print("STEP 8: CREATING GEOMETRIC FEATURES")
print("="*80)

print("Creating geometric features from pass parameters...")

# Elevation categories (max_altitude in degrees)
def categorize_elevation(altitude):
    """Categorize maximum altitude into bins."""
    if pd.isna(altitude):
        return 'unknown'
    elif altitude < 10:
        return 'very_low'
    elif altitude < 30:
        return 'low'
    elif altitude < 60:
        return 'medium'
    elif altitude < 85:
        return 'high'
    else:
        return 'very_high'

df_merged['elevation_category'] = df_merged['max_altitude'].apply(categorize_elevation)

# Duration categories
def categorize_duration(duration):
    """Categorize observation duration."""
    if pd.isna(duration):
        return 'unknown'
    elif duration < 60:  # Less than 1 minute
        return 'very_short'
    elif duration < 300:  # Less than 5 minutes
        return 'short'
    elif duration < 900:  # Less than 15 minutes
        return 'medium'
    elif duration < 1800:  # Less than 30 minutes
        return 'long'
    else:
        return 'very_long'

df_merged['duration_category'] = df_merged['duration_seconds'].apply(categorize_duration)

# Azimuth range (how much the satellite moves in azimuth)
df_merged['azimuth_range'] = abs(df_merged['set_azimuth'] - df_merged['rise_azimuth'])
# Normalize to 0-360 range
df_merged['azimuth_range'] = df_merged['azimuth_range'].apply(lambda x: min(x, 360-x) if not pd.isna(x) else np.nan)

print("✅ Geometric features created:")
print(f"   - Elevation categories: {sorted(df_merged['elevation_category'].unique())}")
print(f"   - Duration categories: {sorted(df_merged['duration_category'].unique())}")
print(f"   - Elevation range: {df_merged['max_altitude'].min():.1f} to {df_merged['max_altitude'].max():.1f} degrees")
print(f"   - Duration range: {df_merged['duration_seconds'].min():.0f} to {df_merged['duration_seconds'].max():.0f} seconds")


STEP 8: CREATING GEOMETRIC FEATURES
Creating geometric features from pass parameters...
✅ Geometric features created:
   - Elevation categories: ['high', 'low', 'medium', 'very_high', 'very_low']
   - Duration categories: ['long', 'medium', 'short', 'very_long']
   - Elevation range: 0.0 to 90.0 degrees
   - Duration range: 180 to 25322 seconds


In [26]:
# ============================================================================
# STEP 9: DATA CLEANING AND VALIDATION
# ============================================================================
print("\n" + "="*80)
print("STEP 9: DATA CLEANING AND VALIDATION")
print("="*80)

initial_rows = len(df_merged)
print(f"Initial rows: {initial_rows:,}")

# 1. Filter out ambiguous observations (no clear success/failure)
df_clean = df_merged[df_merged['target_success'].notnull()].copy()
ambiguous_removed = initial_rows - len(df_clean)
print(f"1. After removing ambiguous status: {len(df_clean):,} rows (removed {ambiguous_removed:,})")

# 2. Remove observations with invalid durations
valid_duration = (df_clean['duration_seconds'] > 10) & (df_clean['duration_seconds'] < 7200)  # 10s to 2 hours
invalid_duration_count = (~valid_duration).sum()
df_clean = df_clean[valid_duration].copy()
print(f"2. After removing invalid durations: {len(df_clean):,} rows (removed {invalid_duration_count:,})")

# 3. Remove invalid elevations (should be between -90 and 90 degrees)
valid_elevation = (df_clean['max_altitude'] >= -90) & (df_clean['max_altitude'] <= 90)
invalid_elevation_count = (~valid_elevation).sum()
df_clean = df_clean[valid_elevation].copy()
print(f"3. After removing invalid elevations: {len(df_clean):,} rows (removed {invalid_elevation_count:,})")

# 4. Remove duplicate observations if any
initial_len = len(df_clean)
df_clean = df_clean.drop_duplicates(subset=['id'], keep='first')
duplicates_removed = initial_len - len(df_clean)
print(f"4. After removing duplicates: {len(df_clean):,} rows (removed {duplicates_removed})")

# Check for remaining missing values in critical columns
print("\n🔍 Missing values in critical columns:")
critical_columns = ['max_altitude', 'duration_seconds', 'target_success', 'hour_of_day', 'station_lat']
missing_summary = []

for col in critical_columns:
    if col in df_clean.columns:
        null_count = df_clean[col].isnull().sum()
        null_pct = null_count / len(df_clean) * 100
        missing_summary.append((col, null_count, null_pct))
        
        if null_count > 0:
            print(f"  ⚠️  {col:20}: {null_count:6,} NULL ({null_pct:5.1f}%)")
        else:
            print(f"  ✅ {col:20}: No missing values")
    else:
        print(f"  ❌ {col:20}: COLUMN NOT FOUND")

print(f"\n📊 Final dataset size: {len(df_clean):,} observations")
print(f"📈 Success rate in cleaned data: {df_clean['target_success'].mean() * 100:.2f}%")

# Analyze station data availability
if 'station_lat' in df_clean.columns:
    station_data_available = df_clean['station_lat'].notnull().sum()
    station_data_pct = station_data_available / len(df_clean) * 100
    print(f"📍 Observations with station coordinates: {station_data_available:,} ({station_data_pct:.1f}%)")


STEP 9: DATA CLEANING AND VALIDATION
Initial rows: 1,000,000
1. After removing ambiguous status: 578,012 rows (removed 421,988)
2. After removing invalid durations: 578,010 rows (removed 2)
3. After removing invalid elevations: 578,010 rows (removed 0)
4. After removing duplicates: 578,010 rows (removed 0)

🔍 Missing values in critical columns:
  ✅ max_altitude        : No missing values
  ✅ duration_seconds    : No missing values
  ✅ target_success      : No missing values
  ✅ hour_of_day         : No missing values
  ✅ station_lat         : No missing values

📊 Final dataset size: 578,010 observations
📈 Success rate in cleaned data: 49.46%
📍 Observations with station coordinates: 578,010 (100.0%)


In [27]:
# ============================================================================
# STEP 10: FEATURE ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("STEP 10: FEATURE ANALYSIS")
print("="*80)

# Group features by category
feature_categories = {
    'Temporal Features': ['hour_of_day', 'day_of_week', 'month', 'year', 
                         'season', 'time_of_day', 'duration_seconds', 'duration_category',
                         'week_of_year', 'day_of_year'],
    'Geometric Features': ['max_altitude', 'rise_azimuth', 'set_azimuth', 
                          'elevation_category', 'azimuth_range'],
    'Station Features': ['station_lat', 'station_lng', 'station_alt', 'horizon', 
                        'grid_square', 'station_status', 'station_name'],
    'Target Variables': ['target_success', 'status', 'status_category', 'vetted_status'],
    'Satellite Info': ['sat_id'],
    'Metadata': ['id', 'start', 'end', 'ground_station_id', 'station_id',
                'waterfall_status', 'archive_url', 'client_version', 'archived', 'experimental']
}

print("📋 Feature inventory by category:")
print("-" * 60)

available_features = []
for category, features in feature_categories.items():
    existing_features = [f for f in features if f in df_clean.columns]
    if existing_features:
        print(f"\n{category} ({len(existing_features)} features):")
        for feature in existing_features:
            dtype = str(df_clean[feature].dtype)
            unique_count = df_clean[feature].nunique()
            available_features.append(feature)
            print(f"  • {feature:25} - {dtype:15} - {unique_count:5} unique values")

print(f"\n📊 Total features available: {len(available_features)}")

# Quick correlation analysis
if 'target_success' in df_clean.columns:
    print("\n🔗 Correlation of top numeric features with target:")
    numeric_features = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove target and ID columns
    exclude_cols = ['target_success', 'id', 'ground_station_id', 'station_id']
    numeric_features = [f for f in numeric_features if f not in exclude_cols]
    
    correlations = []
    for feature in numeric_features[:15]:  # Check first 15 numeric features
        if df_clean[feature].notnull().sum() > len(df_clean) * 0.5:  # At least 50% non-null
            corr = df_clean[feature].corr(df_clean['target_success'])
            if not pd.isna(corr):
                correlations.append((feature, corr))
    
    # Sort by absolute correlation
    correlations.sort(key=lambda x: abs(x[1]), reverse=True)
    
    for feature, corr in correlations[:10]:  # Top 10 correlations
        correlation_strength = "Strong" if abs(corr) > 0.3 else "Moderate" if abs(corr) > 0.1 else "Weak"
        print(f"  {feature:25}: {corr:7.3f} ({correlation_strength})")


STEP 10: FEATURE ANALYSIS
📋 Feature inventory by category:
------------------------------------------------------------

Temporal Features (10 features):
  • hour_of_day               - int32           -    24 unique values
  • day_of_week               - int32           -     7 unique values
  • month                     - int32           -    12 unique values
  • year                      - int32           -     5 unique values
  • season                    - object          -     4 unique values
  • time_of_day               - object          -     4 unique values
  • duration_seconds          - float64         -  1767 unique values
  • duration_category         - object          -     4 unique values
  • week_of_year              - UInt32          -    52 unique values
  • day_of_year               - int32           -   366 unique values

Geometric Features (5 features):
  • max_altitude              - float64         -    91 unique values
  • rise_azimuth              - float64  

In [28]:
# ============================================================================
# STEP 11: SAVE PROCESSED DATASET
# ============================================================================
print("\n" + "="*80)
print("STEP 11: SAVING PROCESSED DATASET")
print("="*80)

# Create output directories
output_dir = "../1_datasets/processed"
os.makedirs(output_dir, exist_ok=True)

# Generate filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
filename = f"four_year_observations_{timestamp}.csv"
full_path = os.path.join(output_dir, filename)

# Save the full dataset
print(f"💾 Saving dataset to: {full_path}")
start_time = time.time()

df_clean.to_csv(full_path, index=False)

save_time = time.time() - start_time
file_size_mb = os.path.getsize(full_path) / (1024 ** 2)

print(f"✅ Dataset saved successfully!")
print(f"   - Rows: {len(df_clean):,}")
print(f"   - Columns: {df_clean.shape[1]}")
print(f"   - File size: {file_size_mb:.2f} MB")
print(f"   - Save time: {save_time:.2f}s")

# Also save a smaller sample for quick EDA
sample_size = min(50000, len(df_clean))
df_sample = df_clean.sample(n=sample_size, random_state=42, replace=False)
sample_path = os.path.join(output_dir, f"four_year_sample_{sample_size}.csv")
df_sample.to_csv(sample_path, index=False)

print(f"\n💾 Sample dataset saved: {sample_path}")
print(f"   - Sample size: {len(df_sample):,} rows")
print(f"   - File size: {os.path.getsize(sample_path) / 1024**2:.2f} MB")


STEP 11: SAVING PROCESSED DATASET
💾 Saving dataset to: ../1_datasets/processed\four_year_observations_20251211_1730.csv
✅ Dataset saved successfully!
   - Rows: 578,010
   - Columns: 39
   - File size: 261.64 MB
   - Save time: 24.29s

💾 Sample dataset saved: ../1_datasets/processed\four_year_sample_50000.csv
   - Sample size: 50,000 rows
   - File size: 22.64 MB


In [29]:
# ============================================================================
# STEP 12: CREATE COMPREHENSIVE SUMMARY REPORT
# ============================================================================
print("\n" + "="*80)
print("STEP 12: CREATING SUMMARY REPORT")
print("="*80)

# Calculate additional statistics
total_years = df_clean['year'].nunique()
total_months = df_clean['month'].nunique()
total_seasons = df_clean['season'].nunique()
station_coverage = df_clean['station_lat'].notnull().sum() / len(df_clean) * 100

# Generate comprehensive summary
summary = f"""
# FOUR-YEAR OBSERVATION DATASET SUMMARY (2021-2025)
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Filename: {filename}

## DATA EXTRACTION
- Time period requested: Last 4 years (from {cutoff_date.strftime('%Y-%m-%d')})
- Latest observation in DB: {latest_date.strftime('%Y-%m-%d %H:%M:%S')}
- Observations in 4-year period: {total_count:,}
- Sample size extracted: {sample_limit:,}
- Final cleaned observations: {len(df_clean):,}

## TEMPORAL COVERAGE
- Date range: {df_clean['start'].min().strftime('%Y-%m-%d')} to {df_clean['start'].max().strftime('%Y-%m-%d')}
- Years represented: {sorted(df_clean['year'].unique())} ({total_years} years)
- Months represented: {sorted(df_clean['month'].unique())} ({total_months} months)
- Seasons represented: {sorted(df_clean['season'].unique())} ({total_seasons} seasons)
- Total time span: {(df_clean['start'].max() - df_clean['start'].min()).days} days

## TARGET VARIABLE
- Success (1): {df_clean['target_success'].sum():,} observations
- Failure (0): {len(df_clean) - df_clean['target_success'].sum():,} observations
- Success rate: {df_clean['target_success'].mean() * 100:.2f}%
- Ambiguous observations removed: {ambiguous_removed:,}

## STATUS CODE DISTRIBUTION (Original)
{status_dist.head(10).to_string()}

## STATION DATA COVERAGE
- Observations with station coordinates: {df_clean['station_lat'].notnull().sum():,}
- Station coverage: {station_coverage:.1f}%
- Unique stations: {df_clean['ground_station_id'].nunique():,}
- Unique stations with location data: {df_clean[df_clean['station_lat'].notnull()]['ground_station_id'].nunique():,}

## FEATURE CATEGORIES
### Temporal Features ({len([f for f in feature_categories['Temporal Features'] if f in df_clean.columns])})
- Hour, day, week, month, year
- Season, time of day categories
- Duration in seconds and categories

### Geometric Features ({len([f for f in feature_categories['Geometric Features'] if f in df_clean.columns])})
- Maximum altitude: {df_clean['max_altitude'].min():.1f} to {df_clean['max_altitude'].max():.1f} degrees
- Elevation categories: {sorted(df_clean['elevation_category'].unique())}
- Duration range: {df_clean['duration_seconds'].min():.0f} to {df_clean['duration_seconds'].max():.0f} seconds

### Station Features ({len([f for f in feature_categories['Station Features'] if f in df_clean.columns])})
- Latitude, longitude, altitude
- Horizon minimum elevation
- Grid square location
- Station status

## DATA QUALITY
- Invalid durations removed: {invalid_duration_count:,}
- Invalid elevations removed: {invalid_elevation_count:,}
- Duplicates removed: {duplicates_removed:,}
- Missing station coordinates: {(len(df_clean) - df_clean['station_lat'].notnull().sum()):,}

## FEATURE CORRELATIONS WITH SUCCESS
Top correlations:
{chr(10).join([f"- {feat}: {corr:.3f}" for feat, corr in correlations[:5]])}

## FILES CREATED
1. `{filename}` - Full cleaned dataset ({len(df_clean):,} rows, {file_size_mb:.1f} MB)
2. `four_year_sample_{sample_size}.csv` - EDA sample ({len(df_sample):,} rows)

## NEXT STEPS
1. Explore dataset in `3_data_exploration/` 
2. Engineer additional features (satellite metadata, weather, etc.)
3. Address missing station data (imputation or feature engineering)
4. Create train/test splits for modeling
5. Begin model development in `4_data_analysis/`
"""

# Save summary to file
summary_path = os.path.join(output_dir, f"four_year_summary_{timestamp}.md")
with open(summary_path, "w") as f:
    f.write(summary)

print(f"📄 Summary report saved: {summary_path}")

# Display key statistics
print("\n" + "="*80)
print("🎯 KEY ACHIEVEMENTS")
print("="*80)

print(f"✅ 4-Year Coverage: {df_clean['year'].min()} to {df_clean['year'].max()}")
print(f"✅ Seasonal Representation: {sorted(df_clean['season'].unique())}")
print(f"✅ Observations: {len(df_clean):,} clean rows")
print(f"✅ Success Rate: {df_clean['target_success'].mean() * 100:.2f}%")
print(f"✅ Features Created: {df_clean.shape[1]} total columns")
print(f"✅ Station Coverage: {station_coverage:.1f}% with coordinates")

# Show sample of data with station info
print("\n" + "="*80)
print("📋 DATASET PREVIEW (Rows with station data)")
print("="*80)

# Show rows that have station data
if 'station_lat' in df_clean.columns and df_clean['station_lat'].notnull().sum() > 0:
    station_rows = df_clean[df_clean['station_lat'].notnull()].head(3)
    preview_cols = ['start', 'duration_seconds', 'max_altitude', 'elevation_category', 
                   'hour_of_day', 'station_lat', 'station_lng', 'target_success']
    available_preview_cols = [col for col in preview_cols if col in station_rows.columns]
    
    if available_preview_cols:
        print(station_rows[available_preview_cols].head(3).to_string())
else:
    print("No station data available in sample")
    print(df_clean[['start', 'duration_seconds', 'max_altitude', 'target_success']].head(3).to_string())


STEP 12: CREATING SUMMARY REPORT
📄 Summary report saved: ../1_datasets/processed\four_year_summary_20251211_1730.md

🎯 KEY ACHIEVEMENTS
✅ 4-Year Coverage: 2021 to 2025
✅ Seasonal Representation: ['fall', 'spring', 'summer', 'winter']
✅ Observations: 578,010 clean rows
✅ Success Rate: 49.46%
✅ Features Created: 39 total columns
✅ Station Coverage: 100.0% with coordinates

📋 DATASET PREVIEW (Rows with station data)
                start  duration_seconds  max_altitude elevation_category  hour_of_day  station_lat  station_lng  target_success
1 2025-10-18 09:45:18             317.0          29.0                low            9     53.91000     27.09000             1.0
4 2025-09-04 12:31:04             180.0          46.0             medium           12     40.16400   -105.09300             0.0
5 2022-08-24 22:39:08             759.0          78.0               high           22     47.25833     16.60462             0.0


In [30]:
# ============================================================================
# COMPLETION
# ============================================================================
print("\n" + "="*80)
print("✅ FOUR-YEAR DATASET PREPARATION COMPLETE!")
print("="*80)

total_time = time.time() - pd.to_datetime('now').timestamp()
print(f"\n⏱️  Total execution time: {abs(total_time):.2f} seconds")

print(f"\n🎯 Next Steps:")
print("   1. Review dataset: 1_datasets/processed/")
print("   2. Check temporal coverage in summary report")
print("   3. Proceed to EDA: 3_data_exploration/")

print(f"\n📁 Your 4-year dataset is ready at: {full_path}")

print("\n" + "="*80)
print("🚀 Ready for Exploration & Modeling!")
print("="*80)


✅ FOUR-YEAR DATASET PREPARATION COMPLETE!

⏱️  Total execution time: 7200.00 seconds

🎯 Next Steps:
   1. Review dataset: 1_datasets/processed/
   2. Check temporal coverage in summary report
   3. Proceed to EDA: 3_data_exploration/

📁 Your 4-year dataset is ready at: ../1_datasets/processed\four_year_observations_20251211_1730.csv

🚀 Ready for Exploration & Modeling!
